In [1]:
import pandas as pd
import numpy as np
import optuna
import dacon_law_class as dlc
from dacon_law_class import SimpleOps as so
import lightgbm as lgb
import re
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from xgboost.sklearn import XGBClassifier as xgb
# import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')


from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

 ___________________________
|                           |
|======== YearDream ========|
|===========================|
|==== DLC Well Imported ====|
|===========================|
|========= BYJASON =========|
|___________________________|



In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
sample_submission = pd.read_csv('./sample_submission.csv')
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [6]:
train
# test
# sample_submission

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1
...,...,...,...,...,...
2473,TRAIN_2473,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al.",Congress amended the Clean Air Act through the...,1
2474,TRAIN_2474,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc.","Alliance Bond Fund, Inc., an investment fund, ...",1
2475,TRAIN_2475,Peguero,United States,"In 1992, the District Court sentenced Manuel D...",0
2476,TRAIN_2476,Immigration and Naturalization Service,St. Cyr,"On March 8, 1996, Enrico St. Cyr, a lawful per...",0


In [4]:
train.info()
print('\n\n\n')
# test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  2478 non-null   object
 1   first_party         2478 non-null   object
 2   second_party        2478 non-null   object
 3   facts               2478 non-null   object
 4   first_party_winner  2478 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 96.9+ KB






In [10]:
df = pd.DataFrame(train['facts'])
df = dlc.law_preprocessor(df, 'facts')
train['facts'] = df['facts']
df = pd.DataFrame(train['first_party'])
df = dlc.law_preprocessor(df, 'first_party')
train['first_party'] = df['first_party']
df = pd.DataFrame(train['second_party'])
df = dlc.law_preprocessor(df, 'second_party')
train['second_party'] = df['second_party']
train_cleansed = train.drop(columns='ID')
train_cleansed


,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,phil amant caidate for public office made tele...,1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,ramon nelson was ridin his bike when he suffer...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",an alabama ate court convicted billy joe mawoo...,1
3,TRAIN_0003,Linkletter,Walker,victor linkletter was convicted in ate court o...,0
4,TRAIN_0004,William Earl Fikes,Alabama,in selma alabama an intruder broke into apartm...,1
...,...,...,...,...,...
2473,TRAIN_2473,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al.",conress ameed clean air act rouh enery policy ...,1
2474,TRAIN_2474,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc.",alliance bo fu inc an invement fu purchased ap...,1
2475,TRAIN_2475,Peguero,United States,in dirict court sentenced manuel peuero to mon...,0
2476,TRAIN_2476,Immigration and Naturalization Service,St. Cyr,enrico cyr lawful permanent resident pled uilt...,0


In [71]:
df = pd.DataFrame(test['facts'])
df = dlc.law_preprocessor(df, 'facts')
test['facts'] = df['facts']
df = pd.DataFrame(test['first_party'])
df = dlc.law_preprocessor(df, 'first_party')
test['first_party'] = df['first_party']
df = pd.DataFrame(test['second_party'])
df = dlc.law_preprocessor(df, 'second_party')
test['second_party'] = df['second_party']
test_cleansed = test.drop(columns='ID')
test_cleansed = test_cleansed.rename(columns={'first_party':'test_first_party_berted', 'second_party':'test_second_party_berted', 'facts':'test_facts_berted'})
test_cleansed

,test_first_party_berted,test_second_party_berted,test_facts_berted
0,salerno,united ates,bail reform act allowed federal courts to deta...
1,milber weiss bershad hynes lerach,lexecon inc,lexecon inc was defeant in class action lawsui...
2,no title federal counications coission et al,fox television ations inc et al,in fox television ations broadca billboa music...
3,harold kaufman,united ates,durin his trial for armed robbery of federally...
4,berer,hanlon,in mairate jude issued warrant auorizin search...
...,...,...,...
1235,haitian centers council inc et al,chris sale actin coissioner iiration naturaliz...,accoin to executive oer no sined by president ...
1236,whitman,american truckin associations inc,section of clean air act requires environmenta...
1237,lia matteo john madian,william barr,lia matteo john madian created plan for utiliz...
1238,washinton ate apple advertisin coission,hunt,in nor carolina boa of ariculture adopted reul...


## BERT

@article{turc2019,
  title={Well-Read Students Learn Better: On the Importance of Pre-training Compact Models},
  author={Turc, Iulia and Chang, Ming-Wei and Lee, Kenton and Toutanova, Kristina},
  journal={arXiv preprint arXiv:1908.08962v2 },
  year={2019}
}


In [18]:
train_facts = pd.DataFrame(train_cleansed['facts'])
train_facts

,facts
0,phil amant caidate for public office made tele...
1,ramon nelson was ridin his bike when he suffer...
2,an alabama ate court convicted billy joe mawoo...
3,victor linkletter was convicted in ate court o...
4,in selma alabama an intruder broke into apartm...
...,...
2473,conress ameed clean air act rouh enery policy ...
2474,alliance bo fu inc an invement fu purchased ap...
2475,in dirict court sentenced manuel peuero to mon...
2476,enrico cyr lawful permanent resident pled uilt...


In [67]:
test_cleansed = pd.DataFrame(test_cleansed['facts'])
test_cleansed

,facts
0,bail reform act allowed federal courts to deta...
1,lexecon inc was defeant in class action lawsui...
2,in fox television ations broadca billboa music...
3,durin his trial for armed robbery of federally...
4,in mairate jude issued warrant auorizin search...
...,...
1235,accoin to executive oer no sined by president ...
1236,section of clean air act requires environmenta...
1237,lia matteo john madian created plan for utiliz...
1238,in nor carolina boa of ariculture adopted reul...


In [19]:
first_party_berted = dlc.auto_tokenizer(train_cleansed, 'first_party')
first_party_berted

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████| 2478/2478 [13:11<00:00,  3.13it/s]


,0
0,"[[tensor(0.0050), tensor(0.0064), tensor(-0.01..."
1,"[[tensor(-0.0039), tensor(0.0268), tensor(-0.0..."
2,"[[tensor(0.0030), tensor(0.0093), tensor(-0.01..."
3,"[[tensor(-0.0152), tensor(-0.0009), tensor(0.0..."
4,"[[tensor(-0.0120), tensor(-0.0100), tensor(0.0..."
...,...
2473,"[[tensor(-0.0206), tensor(-0.0570), tensor(-0...."
2474,"[[tensor(-0.0237), tensor(-0.0229), tensor(-0...."
2475,"[[tensor(-0.0043), tensor(-0.0293), tensor(-0...."
2476,"[[tensor(0.0210), tensor(-0.0031), tensor(-0.0..."


In [31]:
first_party_berted = first_party_berted.rename(columns={0:'first_party_berted'})
first_party_berted.to_csv('./embeddings/first_party_berted.csv', index=False)

In [49]:
second_party_berted = dlc.auto_tokenizer(train_cleansed, 'second_party')
second_party_berted

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████| 2478/2478 [13:11<00:00,  3.13it/s]


,0
0,"[[tensor(-0.0233), tensor(0.0084), tensor(0.01..."
1,"[[tensor(0.0030), tensor(0.0125), tensor(-0.05..."
2,"[[tensor(0.0099), tensor(-0.0199), tensor(-0.0..."
3,"[[tensor(0.0158), tensor(0.0003), tensor(-0.05..."
4,"[[tensor(-0.0077), tensor(0.0062), tensor(-0.0..."
...,...
2473,"[[tensor(0.0304), tensor(-0.0514), tensor(-0.0..."
2474,"[[tensor(0.0282), tensor(0.0164), tensor(-0.02..."
2475,"[[tensor(0.0263), tensor(-0.0167), tensor(-0.0..."
2476,"[[tensor(-0.0208), tensor(-0.0251), tensor(-0...."


In [51]:
second_party_berted = second_party_berted.rename(columns={0:'second_party_berted'})
second_party_berted.to_csv('./embeddings/second_party_berted.csv', index=False)

In [ ]:
facts_berted = dlc.auto_tokenizer(train_cleansed, 'facts')
facts_berted

In [ ]:
facts_berted = embedded_df_1.rename(columns={0:'facts_berted'})
facts_berted.to_csv('./embeddings/facts_berted.csv', index=False)

In [44]:
first_party_berted = pd.read_csv('./embeddings/first_party_berted.csv')

In [52]:
second_party_berted = pd.read_csv('./embeddings/second_party_berted.csv')

In [47]:
facts_berted = pd.read_csv('./embeddings/facts_berted.csv')

In [53]:
all_ready_to_ml = pd.concat([first_party_berted['first_party_berted'], second_party_berted['second_party_berted'], facts_berted['facts_berted'], train_cleansed['first_party_winner']], axis=1)
all_ready_to_ml



,first_party_berted,second_party_berted,facts_berted,first_party_winner
0,"tensor([[ 5.0364e-03, 6.4395e-03, -1.2888e-02...","tensor([[-2.3282e-02, 8.4174e-03, 1.4726e-02...","tensor([[-4.9459e-02, 2.7671e-03, 3.6642e-02...",1
1,"tensor([[-3.9016e-03, 2.6781e-02, -4.4628e-02...","tensor([[ 2.9523e-03, 1.2450e-02, -5.3252e-02...","tensor([[-4.4846e-02, -1.8939e-03, 1.4574e-02...",0
2,"tensor([[ 3.0499e-03, 9.2587e-03, -1.7064e-02...","tensor([[ 9.8794e-03, -1.9934e-02, -5.2907e-02...","tensor([[-4.7123e-02, -6.5828e-03, 1.5028e-02...",1
3,"tensor([[-1.5234e-02, -8.7536e-04, 1.5897e-02...","tensor([[ 1.5804e-02, 2.7409e-04, -5.3871e-02...","tensor([[-8.0872e-02, -2.1650e-02, 9.3573e-03...",0
4,"tensor([[-1.1956e-02, -9.9873e-03, 2.0198e-02...","tensor([[-7.6583e-03, 6.1737e-03, -4.3932e-02...","tensor([[-4.2441e-02, -1.2599e-02, 3.4638e-02...",1
...,...,...,...,...
2473,"tensor([[-2.0591e-02, -5.6972e-02, -3.3879e-02...","tensor([[ 3.0371e-02, -5.1431e-02, -2.3610e-02...","tensor([[-3.8826e-02, -2.5398e-03, 3.1445e-02...",1
2474,"tensor([[-2.3678e-02, -2.2937e-02, -2.2214e-02...","tensor([[ 2.8232e-02, 1.6401e-02, -2.1310e-02...","tensor([[-7.0576e-02, -2.4953e-02, 4.4653e-02...",1
2475,"tensor([[-4.2952e-03, -2.9256e-02, -1.5316e-02...","tensor([[ 2.6328e-02, -1.6656e-02, -2.0892e-02...","tensor([[-7.4061e-02, -1.1856e-02, 1.5850e-02...",0
2476,"tensor([[ 2.1019e-02, -3.1200e-03, -7.0313e-03...","tensor([[-2.0765e-02, -2.5059e-02, -1.7468e-02...","tensor([[-6.7781e-02, -5.1064e-03, 2.9241e-02...",0


In [72]:
test_first_party_berted = dlc.auto_tokenizer(test_cleansed, 'test_first_party_berted')
test_first_party_berted = test_first_party_berted.rename(columns={0:'first_party_berted'})
test_first_party_berted.to_csv('./embeddings/test_first_party_berted.csv', index=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████| 1240/1240 [06:20<00:00,  3.26it/s]


In [73]:
test_second_party_berted = dlc.auto_tokenizer(test_cleansed, 'test_second_party_berted')
test_second_party_berted = test_second_party_berted.rename(columns={0:'second_party_berted'})
test_second_party_berted.to_csv('./embeddings/test_second_party_berted.csv', index=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████| 1240/1240 [06:47<00:00,  3.04it/s]


In [74]:
test_facts_berted = dlc.auto_tokenizer(test_cleansed, 'test_facts_berted')
test_facts_berted = test_facts_berted.rename(columns={0:'test_facts_berted'})
test_facts_berted.to_csv('./embeddings/test_facts_berted.csv', index=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████| 1240/1240 [06:57<00:00,  2.97it/s]


In [75]:
test_first_party_berted = pd.read_csv('./embeddings/test_first_party_berted.csv')
test_second_party_berted = pd.read_csv('./embeddings/test_second_party_berted.csv')
test_facts_berted = pd.read_csv('./embeddings/test_facts_berted.csv')

In [99]:
test_ready_to_ml = pd.concat([test_first_party_berted['first_party_berted'], test_second_party_berted['second_party_berted'], test_facts_berted['test_facts_berted']], axis=1)
test_ready_to_ml



,first_party_berted,second_party_berted,test_facts_berted
0,"tensor([[-6.6722e-03, -8.0011e-02, -1.4580e-02...","tensor([[ 2.6328e-02, -1.6656e-02, -2.0892e-02...","tensor([[-4.1743e-02, -4.1057e-02, 3.7544e-02..."
1,"tensor([[-3.0531e-03, -1.1769e-02, -3.4803e-02...","tensor([[ 1.0108e-02, 1.5938e-03, 6.1594e-03...","tensor([[-6.2039e-02, -9.7709e-03, 3.8074e-02..."
2,"tensor([[-2.5638e-02, -1.6005e-02, -2.1478e-02...","tensor([[ 1.4963e-02, 1.2814e-02, -2.9757e-02...","tensor([[-5.1186e-02, -2.1976e-05, 2.2741e-02..."
3,"tensor([[ 2.8991e-02, 3.6599e-02, -5.9128e-02...","tensor([[ 2.6328e-02, -1.6656e-02, -2.0892e-02...","tensor([[-5.2218e-02, -2.7130e-02, 2.2865e-02..."
4,"tensor([[ 3.6941e-02, -3.9247e-02, -2.6675e-02...","tensor([[ 3.2124e-02, -1.3590e-02, -4.0433e-02...","tensor([[-3.7322e-02, 4.6000e-03, 2.6736e-02..."
...,...,...,...
1235,"tensor([[-1.7606e-02, -2.6507e-02, -3.8503e-02...","tensor([[-3.6597e-02, -2.9370e-02, -8.3051e-03...","tensor([[-5.7088e-02, -6.9000e-04, 2.8725e-02..."
1236,"tensor([[-1.2198e-03, -2.3640e-03, -2.5906e-02...","tensor([[-1.0735e-02, 1.2387e-02, -2.2711e-02...","tensor([[-5.7708e-02, 1.6911e-02, 1.7108e-02..."
1237,"tensor([[ 1.0257e-03, 2.3440e-02, 1.3377e-02...","tensor([[ 1.8597e-02, 2.6318e-02, -3.0361e-02...","tensor([[-5.3087e-02, -1.4558e-02, 2.6369e-02..."
1238,"tensor([[ 1.6344e-03, -8.1419e-04, -1.4695e-02...","tensor([[ 1.8647e-02, 2.5467e-03, -2.3004e-02...","tensor([[-4.9280e-02, -3.1140e-03, 6.9771e-03..."


In [54]:
X = all_ready_to_ml.drop(columns='first_party_winner')
y = all_ready_to_ml['first_party_winner']

In [55]:
label_encoder = LabelEncoder()

X['first_party_berted'] = label_encoder.fit_transform(X['first_party_berted'])
X['second_party_berted'] = label_encoder.fit_transform(X['second_party_berted'])
X['facts_berted'] = label_encoder.fit_transform(X['facts_berted'])

In [81]:
X

,first_party_berted,second_party_berted,facts_berted
0,867,1408,1142
1,1775,556,822
2,575,995,987
3,1296,212,2456
4,1187,1882,710
...,...,...,...
2473,1476,577,549
2474,1544,531,2272
2475,1811,487,2376
2476,343,1344,2178


In [100]:
test_x = test_ready_to_ml
test_x['first_party_berted'] = label_encoder.fit_transform(test_x['first_party_berted'])
test_x['second_party_berted'] = label_encoder.fit_transform(test_x['second_party_berted'])
test_x['test_facts_berted'] = label_encoder.fit_transform(test_x['test_facts_berted'])

In [101]:
test_x = test_x.rename(columns={'test_facts_berted':'facts_berted'})

In [102]:
test_x

,first_party_berted,second_party_berted,facts_berted
0,982,253,173
1,818,4,488
2,774,85,304
3,280,253,322
4,353,305,121
...,...,...,...
1235,668,870,399
1236,580,554,411
1237,6,140,334
1238,100,141,269


In [84]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(1734, 3) (744, 3) (1734,) (744,)


In [85]:
def xgb_objective(trial):
    xgb_params = {
        'objective': 'binary:logistic',
        'eval_metric': 'error',
        'booster': 'gbtree',
        'nthread': trial.suggest_int('nthread', 1, 5),
        'n_estimators' : trial.suggest_int('n_estimators', 25, 1000),
        'max_depth': trial.suggest_int('max_depth', 4, 15),
        'subsample': trial.suggest_uniform('subsample', 0.1, 0.8),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 0.7),
        'lambda': trial.suggest_loguniform('lambda', 0.2, 200),
        'random_state': 42,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 3)
    }
    


    xgb_model = xgb(**xgb_params)
    xgb_model.fit(X_train, y_train)
    xgb_preds = xgb_model.predict(X_val)
    return accuracy_score(y_val, xgb_preds)


In [86]:
xgb_study = optuna.create_study(direction='maximize')
xgb_study.optimize(xgb_objective, n_trials=300)

[I 2023-06-08 01:17:32,930] A new study created in memory with name: no-name-caa1a519-c039-4fca-b2b3-d757061374f7
[I 2023-06-08 01:17:33,036] Trial 0 finished with value: 0.6317204301075269 and parameters: {'nthread': 2, 'n_estimators': 233, 'max_depth': 4, 'subsample': 0.5562540373733703, 'learning_rate': 0.03527002068918461, 'colsample_bytree': 0.5608786898381882, 'lambda': 1.493237090128067, 'min_child_weight': 3}. Best is trial 0 with value: 0.6317204301075269.
[I 2023-06-08 01:17:33,329] Trial 1 finished with value: 0.6626344086021505 and parameters: {'nthread': 5, 'n_estimators': 868, 'max_depth': 11, 'subsample': 0.1947532415283829, 'learning_rate': 0.04032085488762075, 'colsample_bytree': 0.5058806471073195, 'lambda': 59.310227375846154, 'min_child_weight': 2}. Best is trial 1 with value: 0.6626344086021505.
[I 2023-06-08 01:17:33,356] Trial 2 finished with value: 0.6706989247311828 and parameters: {'nthread': 4, 'n_estimators': 70, 'max_depth': 4, 'subsample': 0.24875962731845

[I 2023-06-08 01:17:37,197] Trial 23 finished with value: 0.6706989247311828 and parameters: {'nthread': 2, 'n_estimators': 158, 'max_depth': 10, 'subsample': 0.15052975408749045, 'learning_rate': 0.018771029110402758, 'colsample_bytree': 0.5496044229481146, 'lambda': 66.53599129766528, 'min_child_weight': 2}. Best is trial 2 with value: 0.6706989247311828.
[I 2023-06-08 01:17:37,357] Trial 24 finished with value: 0.6693548387096774 and parameters: {'nthread': 1, 'n_estimators': 330, 'max_depth': 12, 'subsample': 0.31992120315744366, 'learning_rate': 0.012588603150271667, 'colsample_bytree': 0.5159139086021578, 'lambda': 15.828788945234528, 'min_child_weight': 2}. Best is trial 2 with value: 0.6706989247311828.
[I 2023-06-08 01:17:37,435] Trial 25 finished with value: 0.6693548387096774 and parameters: {'nthread': 3, 'n_estimators': 133, 'max_depth': 8, 'subsample': 0.21454191954895707, 'learning_rate': 0.02396158407946959, 'colsample_bytree': 0.5446318440144873, 'lambda': 6.2616848470

[I 2023-06-08 01:17:40,954] Trial 46 finished with value: 0.6706989247311828 and parameters: {'nthread': 4, 'n_estimators': 597, 'max_depth': 12, 'subsample': 0.15411713916398573, 'learning_rate': 0.01570207301489406, 'colsample_bytree': 0.607654363159991, 'lambda': 94.32338732863326, 'min_child_weight': 1}. Best is trial 2 with value: 0.6706989247311828.
[I 2023-06-08 01:17:41,168] Trial 47 finished with value: 0.6706989247311828 and parameters: {'nthread': 3, 'n_estimators': 526, 'max_depth': 6, 'subsample': 0.21629302845830212, 'learning_rate': 0.013487974000069729, 'colsample_bytree': 0.5623541511640069, 'lambda': 34.159865789255505, 'min_child_weight': 2}. Best is trial 2 with value: 0.6706989247311828.
[I 2023-06-08 01:17:41,441] Trial 48 finished with value: 0.6639784946236559 and parameters: {'nthread': 4, 'n_estimators': 759, 'max_depth': 10, 'subsample': 0.14094107823802207, 'learning_rate': 0.024874361882678, 'colsample_bytree': 0.5893073141843804, 'lambda': 18.2802625508193

[I 2023-06-08 01:17:44,169] Trial 69 finished with value: 0.6706989247311828 and parameters: {'nthread': 4, 'n_estimators': 294, 'max_depth': 9, 'subsample': 0.11587538559051565, 'learning_rate': 0.010990943421003378, 'colsample_bytree': 0.5516430571534233, 'lambda': 42.55817155770963, 'min_child_weight': 2}. Best is trial 2 with value: 0.6706989247311828.
[I 2023-06-08 01:17:44,435] Trial 70 finished with value: 0.6518817204301075 and parameters: {'nthread': 3, 'n_estimators': 454, 'max_depth': 7, 'subsample': 0.5703228579036879, 'learning_rate': 0.016764820148595348, 'colsample_bytree': 0.5833651217848357, 'lambda': 14.664186150841672, 'min_child_weight': 3}. Best is trial 2 with value: 0.6706989247311828.
[I 2023-06-08 01:17:44,658] Trial 71 finished with value: 0.6706989247311828 and parameters: {'nthread': 3, 'n_estimators': 668, 'max_depth': 15, 'subsample': 0.1915228300043601, 'learning_rate': 0.013601911079507104, 'colsample_bytree': 0.604895129365676, 'lambda': 107.91120343652

[I 2023-06-08 01:17:47,303] Trial 92 finished with value: 0.6706989247311828 and parameters: {'nthread': 2, 'n_estimators': 38, 'max_depth': 12, 'subsample': 0.21821511898443954, 'learning_rate': 0.012678216746488695, 'colsample_bytree': 0.5361298321531486, 'lambda': 19.7228873537872, 'min_child_weight': 2}. Best is trial 2 with value: 0.6706989247311828.
[I 2023-06-08 01:17:47,352] Trial 93 finished with value: 0.6706989247311828 and parameters: {'nthread': 2, 'n_estimators': 68, 'max_depth': 11, 'subsample': 0.29994713465868805, 'learning_rate': 0.011589658645841947, 'colsample_bytree': 0.5172854393515703, 'lambda': 37.40803430425698, 'min_child_weight': 2}. Best is trial 2 with value: 0.6706989247311828.
[I 2023-06-08 01:17:47,451] Trial 94 finished with value: 0.6706989247311828 and parameters: {'nthread': 3, 'n_estimators': 193, 'max_depth': 11, 'subsample': 0.17961606372697625, 'learning_rate': 0.014398523021069405, 'colsample_bytree': 0.5079422615560979, 'lambda': 24.59405871700

...

[I 2023-06-08 01:18:05,878] Trial 207 finished with value: 0.668010752688172 and parameters: {'nthread': 2, 'n_estimators': 514, 'max_depth': 6, 'subsample': 0.18773301012618265, 'learning_rate': 0.016482631571815884, 'colsample_bytree': 0.5593032481755975, 'lambda': 13.291341449342793, 'min_child_weight': 2}. Best is trial 120 with value: 0.6720430107526881.
[I 2023-06-08 01:18:06,125] Trial 208 finished with value: 0.6666666666666666 and parameters: {'nthread': 4, 'n_estimators': 557, 'max_depth': 14, 'subsample': 0.1297515678102915, 'learning_rate': 0.02725104431729446, 'colsample_bytree': 0.5641157574281943, 'lambda': 25.423107888854116, 'min_child_weight': 1}. Best is trial 120 with value: 0.6720430107526881.
[I 2023-06-08 01:18:06,326] Trial 209 finished with value: 0.668010752688172 and parameters: {'nthread': 3, 'n_estimators': 351, 'max_depth': 10, 'subsample': 0.32738062855806094, 'learning_rate': 0.02469367986584988, 'colsample_bytree': 0.5841444328073404, 'lambda': 31.57188

[I 2023-06-08 01:18:09,584] Trial 230 finished with value: 0.6653225806451613 and parameters: {'nthread': 5, 'n_estimators': 544, 'max_depth': 5, 'subsample': 0.41184537319551046, 'learning_rate': 0.019292626680303657, 'colsample_bytree': 0.5519614396636946, 'lambda': 32.87047566693789, 'min_child_weight': 2}. Best is trial 120 with value: 0.6720430107526881.
[I 2023-06-08 01:18:09,736] Trial 231 finished with value: 0.6706989247311828 and parameters: {'nthread': 1, 'n_estimators': 629, 'max_depth': 8, 'subsample': 0.12751372471151154, 'learning_rate': 0.01003799627634983, 'colsample_bytree': 0.5923968065728735, 'lambda': 172.43391972673462, 'min_child_weight': 1}. Best is trial 120 with value: 0.6720430107526881.
[I 2023-06-08 01:18:10,084] Trial 232 finished with value: 0.6639784946236559 and parameters: {'nthread': 1, 'n_estimators': 626, 'max_depth': 8, 'subsample': 0.47024886491433215, 'learning_rate': 0.0104922480525477, 'colsample_bytree': 0.6365114903871768, 'lambda': 15.708215

[I 2023-06-08 01:18:13,564] Trial 253 finished with value: 0.6706989247311828 and parameters: {'nthread': 5, 'n_estimators': 58, 'max_depth': 6, 'subsample': 0.16942716624374357, 'learning_rate': 0.011276621912263874, 'colsample_bytree': 0.5197477639232563, 'lambda': 53.042430954894414, 'min_child_weight': 2}. Best is trial 120 with value: 0.6720430107526881.
[I 2023-06-08 01:18:13,733] Trial 254 finished with value: 0.6706989247311828 and parameters: {'nthread': 3, 'n_estimators': 364, 'max_depth': 10, 'subsample': 0.2837021840939583, 'learning_rate': 0.01999585589736823, 'colsample_bytree': 0.6048622538257943, 'lambda': 79.52419853903861, 'min_child_weight': 2}. Best is trial 120 with value: 0.6720430107526881.
[I 2023-06-08 01:18:14,016] Trial 255 finished with value: 0.6706989247311828 and parameters: {'nthread': 4, 'n_estimators': 646, 'max_depth': 8, 'subsample': 0.3164794311811773, 'learning_rate': 0.01365298412529285, 'colsample_bytree': 0.5090163471344153, 'lambda': 100.897845

[I 2023-06-08 01:18:17,570] Trial 276 finished with value: 0.6706989247311828 and parameters: {'nthread': 1, 'n_estimators': 33, 'max_depth': 11, 'subsample': 0.1016658068692884, 'learning_rate': 0.0194970635129158, 'colsample_bytree': 0.514883822055735, 'lambda': 50.890533556573196, 'min_child_weight': 3}. Best is trial 120 with value: 0.6720430107526881.
[I 2023-06-08 01:18:17,735] Trial 277 finished with value: 0.6706989247311828 and parameters: {'nthread': 2, 'n_estimators': 495, 'max_depth': 14, 'subsample': 0.15061360008671146, 'learning_rate': 0.011685374191683574, 'colsample_bytree': 0.5656926244153243, 'lambda': 147.7239191874773, 'min_child_weight': 1}. Best is trial 120 with value: 0.6720430107526881.
[I 2023-06-08 01:18:17,984] Trial 278 finished with value: 0.668010752688172 and parameters: {'nthread': 5, 'n_estimators': 532, 'max_depth': 7, 'subsample': 0.2292082013935545, 'learning_rate': 0.024238592837504697, 'colsample_bytree': 0.581540816481638, 'lambda': 47.443915374

[I 2023-06-08 01:18:21,924] Trial 299 finished with value: 0.6706989247311828 and parameters: {'nthread': 1, 'n_estimators': 589, 'max_depth': 10, 'subsample': 0.12311552363886395, 'learning_rate': 0.013911240073140211, 'colsample_bytree': 0.5445883806407259, 'lambda': 50.913324274021704, 'min_child_weight': 1}. Best is trial 120 with value: 0.6720430107526881.


In [87]:
print('Number of finished XGB trials: {}'.format(len(xgb_study.trials)))
print('XGB Best trial:')
xgb_trial = xgb_study.best_trial

print('  Value: {}'.format(xgb_trial.value))
print('  Params: ')

for key, value in xgb_trial.params.items():
    print('    {}: {}'.format(key, value))

Number of finished XGB trials: 300
XGB Best trial:
  Value: 0.6720430107526881
  Params: 
    nthread: 3
    n_estimators: 541
    max_depth: 11
    subsample: 0.24919214519121702
    learning_rate: 0.014080320456224983
    colsample_bytree: 0.5844754141851671
    lambda: 20.970246761043718
    min_child_weight: 3


In [88]:
xgb_params = xgb_trial.params
XGB = xgb(**xgb_params)
XGB.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5844754141851671, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=20.970246761043718,
              learning_rate=0.014080320456224983, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=11, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=541, n_jobs=None, nthread=3, num_parallel_tree=None, ...)

In [89]:
XGB_pred = XGB.predict(X_val)

In [90]:
# print("-- LGB --")
# print("Train ACC : %.3f" % accuracy_score(y_train, lgb_model.predict(X_train)))
# print("Val ACC : %.3f" % accuracy_score(y_val, lgb_model.predict(X_val)))

print("\n-- XGB --")
print("Train ACC : %.3f" % accuracy_score(y_train, XGB.predict(X_train)))
print("Val ACC : %.3f" % accuracy_score(y_val, XGB.predict(X_val)))


-- XGB --
Train ACC : 0.672
Val ACC : 0.671


In [103]:
X_test = pd.get_dummies(data=test_x)
print(X_test)
XGB_pred = XGB.predict(test_x)

      first_party_berted  second_party_berted  facts_berted
0                    982                  253           173
1                    818                    4           488
2                    774                   85           304
3                    280                  253           322
4                    353                  305           121
...                  ...                  ...           ...
1235                 668                  870           399
1236                 580                  554           411
1237                   6                  140           334
1238                 100                  141           269
1239                 962                  661           494

[1240 rows x 3 columns]


'''
print(classification_report(y_val, lgb_preds))
print(classification_report(y_val, XGB_pred))
'''

In [105]:
# LGB_submission = pd.read_csv('./sample_submission.csv')
# LGB_pred = pd.DataFrame(lgb_preds.astype(int))
# LGB_submission['first_party_winner'] = LGB_pred
# LGB_submission.to_csv("./LGB_tryout_submission.csv", index=False)

XGB_submission = pd.read_csv('./sample_submission.csv')
XGB_pred = pd.DataFrame(XGB_pred.astype(int))
XGB_submission['first_party_winner'] = XGB_pred
XGB_submission.to_csv("./Bert_XGB_submission.csv", index=False)

In [ ]:
combined_df = pd.concat([df_part_1, df_part_2, df_part_3, df_part_4])

In [ ]:
combined_df

In [ ]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-uncased", add_pooling_layer=False, output_hidden_states=True, output_attentions=True)
model.num_parameters()

In [ ]:
# 3D to 2D

attention_mask_df = dlc.tensor_2_2d(train_bert_tokenized, 0)
input_ids_df = dlc.tensor_2_2d(train_bert_tokenized, 1)
token_type_ids_df = dlc.tensor_2_2d(train_bert_tokenized, 2)

attention_mask_df.info()
print('\n _______________________________ \n')
input_ids_df.info()
print('\n _______________________________ \n')
token_type_ids_df.info()


In [ ]:
# attention_mask_df.info()
attention_mask_df
# input_ids_df.info()
# input_ids_df
# token_type_ids_df.info()
# token_type_ids_df


In [ ]:
temp = pd.DataFrame()
temp = pd.concat([train_cleansed['ID'], attention_mask_df], axis=1)
temp = pd.concat([temp, input_ids_df], axis=1)
train_BertToken_df = pd.concat([temp, token_type_ids_df], axis=1)
train_BertToken_df


In [ ]:
tBTdf = so.right_merger(train_cleansed, train_BertToken_df, 0)
tBTdf